# TP noté


**Ce TP est à rendre par email pour le vendredi 03 mai, dernier délai.**

Vous pouvez rendre :
- ou bien un fichier au format **.ipynb**, par exemple en téléchargeant puis en éditant ce sujet (voir icône de téléchargement en haut à droite), ou en créant votre propre fichier **.ipynb**
- ou bien un fichier de script au format **.py** ou **.sage**

## Présentation du sujet

Dans ce TP, l'objectif est **d'implanter l'algorithme de décodage de Berlekamp-Welch** pour les **codes de Reed-Solomon**. Cet algorithme permet corriger un nombre d'erreurs allant jusqu'au rayon de décodage unique $\lfloor \frac{d-1}{2}\rfloor$.

Rappelons qu'un code de Reed-Solomon est défini par un ensemble de points d'évaluation deux à deux distincts ${\bf a} = (a_1, a_2, \dots, a_n) \in \mathbb{F}_q^2$ et par une dimension $k$, comme l'ensemble :

$$
{\rm RS}_k({\bf a}) = \{ (f(a_1), f(a_2), \dots, f(a_n)) \mid f \in \mathbb{F}_q[x], {\rm deg}(f) < k \}\,.
$$

Ces codes sont MDS, c'est-à-dire que la distance minimale du code atteint la borne de Singleton : $d = n-k+1$. Ainsi, le rayon de décodage unique d'un code de Reed-Solomon est $t := \lfloor \frac{d-1}{2}\rfloor = \lfloor \frac{n-k}{2}\rfloor$.

**L'algorithme de Berlekamp-Welch** est le suivant :

* **Entrée :** un mot de code bruité ${\bf y} = {\bf c} + {\bf e} \in \mathbb{F}_q^n$, où ${\bf c} \in {\rm RS}_k({\bf a})$ et  ${\bf e}  \in \mathbb{F}_q^n$ est de poids $\le t$.
* **Sortie :** le polynôme $f(x) \in \mathbb{F}_q[x]$ de degré $< k$ tel que  ${\bf c} = (f(a_1), f(a_1), \dots, f(a_n))$.

1. Construire la matrice 

$$
{\bf M} =
\begin{pmatrix}
y_1 & y_1 a_1 & y_1 a_1^2 & \dots & y_1 a_1^t & -1 & - a_1 & -a_1^2 &  \dots & -a_1^{k+t-1} \\
y_2 & y_2 a_2 & y_2 a_2^2 & \dots & y_2 a_2^t & -1 & - a_2 & -a_2^2 &  \dots & -a_2^{k+t-1} \\
y_3 & y_3 a_3 & y_3 a_3^2 & \dots & y_3 a_3^t & -1 & - a_3 & -a_3^2 &  \dots & -a_3^{k+t-1} \\
\vdots & \vdots & \vdots & & \vdots & \vdots & & \vdots \\
\vdots & \vdots & \vdots & & \vdots & \vdots & & \vdots \\
\vdots & \vdots & \vdots & & \vdots & \vdots & & \vdots \\
\vdots & \vdots & \vdots & & \vdots & \vdots & & \vdots \\
y_n & y_n a_n & y_n a_n^2 & \dots & y_n a_n^t & -1 & - a_n & -a_n^2 &  \dots & -a_n^{k+t-1} \\
\end{pmatrix}
$$

2. Trouver une solution non-nulle ${\bf s}$ au système ${\bf M s} = {\bf 0}$, formé de $n$ équations et $k+2t$ inconnues.

3. Décomposer ${\bf s} = ({\bf u}, {\bf v})$ en deux vecteurs de coefficients, où ${\bf u} \in \mathbb{F}_q^{t+1}$ et ${\bf v} \in  \mathbb{F}_q^{k+t}$.

4. Construire les polynômes $U(x) = \sum u_i x^i$ et $V(x) = \sum v_i x^i$, et vérifier que $U(x)$ divise bien $V(x)$ (si ce n'est pas le cas, **retourner** une erreur).

5. **Retourner** le polynôme $f(x) = \frac{V(x)}{U(x)}$.


Dans ce TP, pour simplifier les choses, nous allons choisir des paramètres relativement petits : le corps fini sera $\mathbb{F}_{19}$, la longueur $n = 14$ et la dimension $k = 6$.

## Partie 1. Construction d'un mot de code, ajout des erreurs



**Question 1.-** Construire :
- le corps fini $\mathbb{F}_{19}$ et le stocker dans une variable `F`,
- l'anneau de polynômes $\mathbb{F}_{19}[X]$ et le stocker dans une variable `R`,
- l'indéterminée $X$ de l'anneau $\mathbb{F}_{19}[X]$ et la stocker dans une variable `X`,
- deux entiers $n = 14$ et $k = 6$ et les stocker dans des variables du même nom,
- la liste de points d'évaluation deux à deux distincts ${\bf a} = (1, 2, 3, 4, \dots, 12, 13, 14) \in \mathbb{F}_{19}^n$, et la stocker dans une variable `points`.

In [257]:
# Votre réponse ici


F=GF(19)
R = PolynomialRing(F, "X")
X = R.gen()
n=14
k=6
t=int(math.floor((n-k)/2))
print(t)
points=[i for i in range(1,15)]

4


**Question 2.-** Écrire une fonction `polynome_aleatoire()` qui retourne un polynôme tiré uniformément dans l'ensemble des polynômes de $\mathbb{F}_{19}[X]$ de degré $< k$.

In [258]:
# Votre réponse ici
from random import choice

def polynome_aleatoire():
    L=[]
    for i in range(0,k):
        L.append(F.random_element())
        
    Polynome=0
    for i in range(0,len(L)):
        Polynome=Polynome+L[i]*X**i
        
    return Polynome


polynome_aleatoire()

12*X^5 + 18*X^4 + 6*X^3 + 15*X^2 + 5*X + 1

**Question 3.-** Écrire une fonction `encode(message)` qui prend en entrée un polynôme `message` de degré $< k$, et qui retourne le mot du code de Reed-Solomon ${\rm RS}_k({\bf a})$ associé. ce mot de code pourra être écrit comme une liste ou comme un élément de la classe `vector`.

In [259]:
# Votre réponse ici
def encode(message):
    if message.degree()>k :
        print("erreur")
    else:
        L=message.coefficients(sparse=False)
        liste_encode=[]
        for i in range(0,len(points)):
            somme=0
            for j in range(0,len(L)):
                           somme=somme+(points[i]**j)*L[j]
            liste_encode.append(somme)
            
        return liste_encode


exemple_mot_de_code=encode(polynome_aleatoire())
exemple_mot_de_code
    
    

[7, 15, 18, 18, 12, 7, 16, 16, 1, 16, 1, 15, 4, 6]

**Question 4.-** Écrire une fonction `interpole(mot)` qui prend en entrée un mot de code `mot` du code de Reed-Solomon ${\rm RS}_k({\bf a})$, et retrouve le polynôme qui a produit ce mot de code. En quelque sorte, votre fonction `interpole` doit être la fonction inverse de la fonction `encode` écrite à la question précédente.

Pour répondre à cette question on pourra utiliser la méthode `lagrange_polynomial` de la classe `PolynomialRing`, dont la documentation est fournie [ici](https://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/polynomial_ring.html#sage.rings.polynomial.polynomial_ring.PolynomialRing_field.lagrange_polynomial).

In [260]:
# Votre réponse ici

def interpole(mot):
    couple = [[] for _ in range(len(mot))]
    for i in range(0,len(mot)):
        couple[i]=[points[i],mot[i]]
    return R.lagrange_polynomial(couple)



interpole(exemple_mot_de_code)
    

12*X^5 + 9*X^4 + 17*X^3 + 6*X^2 + 10*X + 10

**Question 5.-** Écrire une fonction `introduit_erreurs(mot, t)` qui prend en entrée un mot de code `mot` et un entier naturel `t`, et qui retourne une version bruitée du mot de code `mot` avec `t` erreurs.



In [261]:
# Votre réponse ici
from random import choice

def introduit_erreurs(mot,t):
    mot_bruite=[]
    liste_localisation_erreur=[]
    
    while len((set(liste_localisation_erreur)))<t :
        localisation_erreur=randint(0,len(mot)-1)
        liste_localisation_erreur.append(localisation_erreur)
        
        #mot[localisation_erreur]=mot[localisation_erreur]+erreur 
        
        #REMARQUE :J'ai peur que la ligne en commentaire ne fonctionne pas avec tous les exmeples, en effet de mémoire quand on remplace/change des élments d'une liste sans pop ou append c'est problématique.
        
        #Et a cause de ça je vais devoir faire un truc compliqué...
        
    liste_localisation_erreur= sorted(set(liste_localisation_erreur)) #Je trie dans l'ordre croissant parce que ça va m'aider pour mieux m' 
    
    if len(liste_localisation_erreur)<len(mot):
        i=0
        while i < len(liste_localisation_erreur)-1:

            difference = liste_localisation_erreur[i+1]-liste_localisation_erreur[i] #Je fais du "padding" avec des a entre les positions des erreurs qui ne sont pas conséqutives 
            if difference > 1:
                liste_localisation_erreur[i+1:i+1] = ["a"]*(difference - 1) #exemple : [0,1,2,5]. Pour remplir à la position 3 et 4 avec des "a" je rempli pas à la position 2 mais a la position 2+1 d'où le [i+1 , et difference -1 parce qu'on rempli que entre 3 et 5, on ne veut pas toucher 5. On s'arrête un pas avant. 
   
                i += difference-1 #Pour sauter les cases parce qu'on a la certitude de les avoir rendu conséqutives
            i += 1
        
        if liste_localisation_erreur[0]!=0:
            for i in range(0,liste_localisation_erreur[0]): #Cas où [3,4] pour transformer en [0,1,2]
                liste_localisation_erreur.insert(0, "a")
            
        if liste_localisation_erreur[-1]<len(mot): 
            for i in range(0,(len(mot)-1)-liste_localisation_erreur[-1]):#Cas où le mot est [1,2,3,4,5] mais que l'erreur est [a,a,2]. On transforme en [a,a,2,a,a,a]. len(mot)-1 c'est parce que dernier_element=longueur_liste-1 pour toute les listes
                liste_localisation_erreur.append("a")
    
    for i in range(0,len(mot)):
        
        if i==liste_localisation_erreur[i] :
            erreur=randint(1,18) #aucun interet de prendre une erreur à 0

            mot_bruite.append(mot[i]+erreur) #A la positin i de l'erreur j'additionne une erreur.
        else :
            mot_bruite.append(mot[i])  #Sinon à la position i je copie l'élement 
    return mot_bruite
            
        
print(exemple_mot_de_code)        
introduit_erreurs(exemple_mot_de_code,5)
        
        
    
        

[7, 15, 18, 18, 12, 7, 16, 16, 1, 16, 1, 15, 4, 6]


[13, 15, 18, 18, 10, 7, 16, 16, 3, 16, 1, 11, 15, 6]

## Partie 2. L'algorithme de décodage



**Question 6.-** Écrire une fonction `construit_matrice(y)` qui prend en entrée un mot bruité `y`, et qui construit la matrice ${\bf M}$ du système linéaire de décodage (**étape 1** de l'algorithme).

In [262]:
"""# Votre réponse ici
s=[1,2,3,4,5,6,7]
premier=s[:2]
print(premier)
dernier= s[2:]
print(dernier)
"""
# Votre réponse ici
import math

def construit_matrice(y) :
    t=int(math.floor((n-k)/2))
    Ligne=[[] for i in range(0,14)]
    for i in range(0,n):
        for j in range(0,t):
            
            Ligne[i].append(y[i]*(pow(points[i],j,19)))
        for j in range(0,k+t):
            Ligne[i].append(-(pow(points[i],j,19))) #
            
        Ligne[i]=vector(F, Ligne[i])
        
    #matrice = np.array(Ligne)
    M= Matrix(Ligne)
    return M

y=encode(polynome_aleatoire())
M=construit_matrice(y)
M


[17 17 17 17 18 18 18 18 18 18 18 18 18 18]
[14  9 18 17 18 17 15 11  3  6 12  5 10  1]
[17 13  1  3 18 16 10 11 14  4 12 17 13  1]
[ 7  9 17 11 18 15  3 12 10  2  8 13 14 18]
[ 2 10 12  3 18 14 13  8  2 10 12  3 15 18]
[11  9 16  1 18 13  2 12 15 14  8 10  3 18]
[ 7 11  1  7 18 12  8 18 12  8 18 12  8 18]
[14 17  3  5 18 11 12  1  8  7 18 11 12  1]
[ 4 17  1  9 18 10 14 12 13  3  8 15  2 18]
[ 3 11 15 17 18  9 14  7 13 16  8  4  2  1]
[ 7  1 11  7 18  8 12 18  8 12 18  8 12 18]
[12 11 18  7 18  7  8  1 12 11 18  7  8  1]
[ 6  2  7 15 18  6  2  7 15  5  8  9  3  1]
[18  5 13 11 18  5 13 11  2  9 12 16 15  1]

**Question 7.-** Écrire une fonction `berlekamp_welch(y)` qui décode le mot buité `y` grâce à l'algorithme de Berlekamp-Welch. Cette fonction retournera donc un polynôme.

In [263]:
# Votre réponse ici
def berlekamp_welch(y):
    #=M.right.kernel()
    
    M=construit_matrice(y)
  
    noyau = M.right_kernel()
    
    if len(noyau)== 1:
        raise ValueError("La matrice n'a qu'un seul élément dans son noyau")  #Je fais ça parce que sinon on ne pourra pas faire la division mentionné dans l'étape 4 de l'algorithme, (dans le noyau on a toujours 0,..,0 donc si il y'a qu'un unique element dans le noyau c'est forcement lui )
    i=1
    vecteur_noyau=noyau[1]
    while vecteur_noyau[:t]==[0 for i in range (0,t)]  :
        vecteur_noyau= M.right_kernel()[i]
        i=i+1
    reste=1
    print('len',len(noyau))
    for j in range(i,len(noyau)):
        Coeff_Polynome_localisateur=vecteur_noyau[:t]
        Coeff_Polynome_N=vecteur_noyau[t:]
        Polynome_localisateur=0
        for i in range(0,t-1):
            Polynome_localisateur=Polynome_localisateur+Coeff_Polynome_localisateur[i]*(X**i) #variable X=commande"
            Polynome_N=0
        for i in range(0,len(Coeff_Polynome_N)):
            Polynome_N=Polynome_N+Coeff_Polynome_N[i]*(X**i)

        quotient,reste=Polynome_N.quo_rem(Polynome_localisateur)
        if reste==0 :
            break
        print(j)
        print("Le reste est:",reste)
        vecteur_noyau= M.right_kernel()[j]

    if reste==0:    #Conditon pour le dernier test si on a finit la boucle for
        f=quotient
        L=[]
        for i in range(0,len(points)):
            L.append(f.substitute(X=points[i]))
            

        return f,L

    else :
        print("Erreur")


y=encode(polynome_aleatoire())
mot_bruite=introduit_erreurs(y,3)
print(mot_bruite)
berlekamp_welch(mot_bruite)

[8, 10, 15, 7, 0, 13, 2, 1, 15, 16, 17, 2, 1, 12]
len 19
1
Le reste est: 6*X + 13
2
Le reste est: 6*X + 13
3
Le reste est: 12*X + 7
4
Le reste est: 18*X + 1
5
Le reste est: 5*X + 14
6
Le reste est: 11*X + 8
7
Le reste est: 17*X + 2
8
Le reste est: 4*X + 15
9
Le reste est: 10*X + 9
10
Le reste est: 16*X + 3
11
Le reste est: 3*X + 16
12
Le reste est: 9*X + 10
13
Le reste est: 15*X + 4
14
Le reste est: 2*X + 17
15
Le reste est: 8*X + 11
16
Le reste est: 14*X + 5
17
Le reste est: X + 18
18
Le reste est: 7*X + 12
Erreur


**Question 8.-** Tester l'algorithme d'interpolation de la **question 4** et l'algorithme de décodage de la **question 7** sur plusieurs exemples tirés aléatoirement.

In [264]:
# Votre réponse ici

def test_interpolation(points,nombre_de_test):
    L=[]
    Compteur=0
    for i in range(0,nombre_de_test):
        L=[]
        mot=encode(polynome_aleatoire())
        
        polynome=interpole(mot)
        
        for j in range(0,len(points)):
            L.append(polynome.substitute(X=points[j]))
            
        if (L==mot) :
            Compteur=Compteur+1
    if Compteur==nombre_de_test :
        print("La fonction d'interpolation est bonne")
    elif Compteur>(nombre_de_test)/2 :
        print("La fonction d'interpolation n'est pas assez bonne")
        
    else :
        print("La fonction d'interpolation n'est pas bonne")
        
test_interpolation(points,10)
    
def test_decodage(points):
    for i in range(0,5):
        mot_de_code=encode(polynome_aleatoire())
        mot_bruite=introduit_erreurs(mot_de_code,t)
        print(mot_bruite)
        message=berlekamp_welch(mot_bruite)
        
        
        if(message.degree()>=k):
            print("erreur")
        else:
            
            for j in range(0,len(points)):
                L.append(message.subs(X,points[j]))
            distance_entre_mot_mot_bruite=0
            for g in range(0,len(mot_bruite)):
                if L[g]==points[g] :
                    distance_entre_mot_mot_bruite=+1
                    
            if distance_entre_mot_mot_bruite<=int(math.floor((n-k)/2)) :
                print("Le test",i, "a fonctionner" )

mot_bruite=introduit_erreurs(y,1)
test_decodage(points)
            

La fonction d'interpolation est bonne
[7, 9, 17, 11, 15, 4, 11, 18, 2, 14, 3, 1, 11, 18]


ValueError: La matrice n'a qu'un seul élément dans son noyau